Licensed under the MIT License.

Copyright (c) 2021-2031. All rights reserved.

# Model Selection with AutoKeras

* It will try to find thr optimal neural network architecture & hyperparameters for you. 
* Totorials: https://autokeras.com/tutorial/overview/

In [39]:
import autokeras
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, balanced_accuracy_score

### Regression
* AutoKeras `StructuredDataRegressor`: https://autokeras.com/structured_data_regressor/
* Too slow when the dataset has either large dimensions or large number of records, so here removed many features

In [57]:
df = pd.read_pickle('../luigi_pipeline/output/preprocessed_data.pkl')
print(df.shape)

df.head()

(693861, 22)


,Store,Date,Year,Month,Quarter,Customers_larger_than_3000,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,...,Promo2SinceWeek,Promo2SinceYear,PromoInterval,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,2015-07-31,2015,7,941,0.0,2,0,1270.0,9,...,-1,-1,0,5,5263,555,1,1,0,1
1,1,2015-07-30,2015,7,940,0.0,2,0,1270.0,9,...,-1,-1,0,4,5020,546,1,1,0,1
2,1,2015-07-29,2015,7,939,0.0,2,0,1270.0,9,...,-1,-1,0,3,4782,523,1,1,0,1
3,1,2015-07-28,2015,7,938,0.0,2,0,1270.0,9,...,-1,-1,0,2,5011,560,1,1,0,1
4,1,2015-07-27,2015,7,937,0.0,2,0,1270.0,9,...,-1,-1,0,1,6102,612,1,1,0,1


In [58]:
cat_cols = [col for col in df.select_dtypes(include='category').columns if col != 'Year']
df.drop(cat_cols, axis=1, inplace=True)

df.head()

,Date,Year,Customers_larger_than_3000,CompetitionDistance,Sales,Customers
0,2015-07-31,2015,0.0,1270.0,5263,555
1,2015-07-30,2015,0.0,1270.0,5020,546
2,2015-07-29,2015,0.0,1270.0,4782,523
3,2015-07-28,2015,0.0,1270.0,5011,560
4,2015-07-27,2015,0.0,1270.0,6102,612


In [59]:
# train, test split for df
train_df = df.loc[df['Year'].astype(str) < '2015']
test_df = df.loc[df['Year'].astype(str) == '2015']

y_train, y_test = train_df['Sales'], test_df['Sales']
X_train, X_test = train_df.drop(['Sales', 'Date', 'Year'], axis=1), test_df.drop(['Sales', 'Date', 'Year'], axis=1)

X_train.reset_index(inplace=True, drop=True)
X_test.reset_index(inplace=True, drop=True)
y_train.reset_index(inplace=True, drop=True)
y_test.reset_index(inplace=True, drop=True)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
X_train.head()

(532529, 3) (161332, 3) (532529,) (161332,)


,Customers_larger_than_3000,CompetitionDistance,Customers
0,0.0,1270.0,327
1,0.0,1270.0,703
2,0.0,1270.0,700
3,0.0,1270.0,0
4,0.0,1270.0,684


In [60]:
ak_regressor = autokeras.StructuredDataRegressor(max_trials=3, tuner='bayesian')
ak_regressor.fit(X_train, y_train, epochs=3)

Trial 3 Complete [00h 07m 41s]
val_loss: 7056720.0

Best val_loss So Far: 7056720.0
Total elapsed time: 00h 44m 37s
INFO:tensorflow:Oracle triggered exit
Epoch 1/3
16642/16642 [==============================] - 111s 7ms/step - loss: 9390212.0000 - mean_squared_error: 9390212.0000
Epoch 2/3
16642/16642 [==============================] - 120s 7ms/step - loss: 8155811.0000 - mean_squared_error: 8155811.0000
Epoch 3/3
16642/16642 [==============================] - 123s 7ms/step - loss: 7367745.5000 - mean_squared_error: 7367745.5000
INFO:tensorflow:Assets written to: .\structured_data_regressor\best_model\assets


In [61]:
y_pred_regression = ak_regressor.predict(X_test)
print(r2_score(y_test, y_pred_regression.flatten().astype(np.float)))

5042/5042 [==============================] - 14s 3ms/step
-0.2587499301929723


In [62]:
keras_export_regression = ak_regressor.export_model()
keras_export_regression.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
multi_category_encoding (Mul (None, 3)                 0         
_________________________________________________________________
normalization (Normalization (None, 3)                 7         
_________________________________________________________________
dense (Dense)                (None, 1024)              4096      
_________________________________________________________________
re_lu (ReLU)                 (None, 1024)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0     

### Classification
* AutoKeras `StructuredDataClassifier`: https://autokeras.com/structured_data_classifier/

In [10]:
df30 = pd.read_csv('../../crystal_ball/data_collector/structured_data/leaf.csv')

y30 = df30['species']
X30 = df30.drop('species', axis=1)

X_train30, X_test30, y_train30, y_test30 = train_test_split(X30, y30, test_size=0.2,
                                               random_state=10, shuffle=True, stratify=y30)

X_train30.reset_index(inplace=True, drop=True)
X_test30.reset_index(inplace=True, drop=True)
y_train30.reset_index(inplace=True, drop=True)
y_test30.reset_index(inplace=True, drop=True)

print(X_train30.shape, X_test30.shape, y_train30.shape, y_test30.shape)
print(y_train30.nunique(), y_test30.nunique())

(272, 15) (68, 15) (272,) (68,)
30 30


In [11]:
ak_classifier = autokeras.StructuredDataClassifier(max_trials=10)
ak_classifier.fit(X_train30, y_train30, epochs=100)

Trial 10 Complete [00h 00m 39s]
val_accuracy: 0.75

Best val_accuracy So Far: 0.9375
Total elapsed time: 00h 06m 38s
INFO:tensorflow:Oracle triggered exit
Epoch 1/100
9/9 [==============================] - 1s 8ms/step - loss: 3.4849 - accuracy: 0.0110
Epoch 2/100
9/9 [==============================] - 0s 8ms/step - loss: 3.3863 - accuracy: 0.0221
Epoch 3/100
9/9 [==============================] - 0s 12ms/step - loss: 3.3047 - accuracy: 0.0515
Epoch 4/100
9/9 [==============================] - 0s 12ms/step - loss: 3.2301 - accuracy: 0.0882
Epoch 5/100
9/9 [==============================] - 0s 12ms/step - loss: 3.1576 - accuracy: 0.1213
Epoch 6/100
9/9 [==============================] - 0s 11ms/step - loss: 3.0842 - accuracy: 0.1507
Epoch 7/100
9/9 [==============================] - 0s 12ms/step - loss: 3.0065 - accuracy: 0.1875
Epoch 8/100
9/9 [==============================] - 0s 12ms/step - loss: 2.9225 - accuracy: 0.2132
Epoch 9/100
9/9 [==============================] - 0s 8ms/step 

In [21]:
y_pred_classification = ak_classifier.predict(X_test30)
print(balanced_accuracy_score(y_test30, y_pred_classification.flatten().astype(np.float)))

3/3 [==============================] - 0s 6ms/step
0.8444444444444446


In [22]:
keras_export_classification = ak_classifier.export_model()
keras_export_classification.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 15)]              0         
_________________________________________________________________
multi_category_encoding (Mul (None, 15)                0         
_________________________________________________________________
normalization (Normalization (None, 15)                31        
_________________________________________________________________
dense (Dense)                (None, 32)                512       
_________________________________________________________________
re_lu (ReLU)                 (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
re_lu_1 (ReLU)               (None, 32)                0     